In [2]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader, random_split
from PIL import Image
import io
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
import math
import os
from tqdm import tqdm
import numpy as np

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 資料預處理
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)

train_size = int(0.8 * len(dataset))
valid_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(
    dataset, [train_size, valid_size, test_size]
)

# 資料載入器設定
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# 改進的JPEG壓縮函數
def jpeg_compress(x, quality):
    """執行JPEG壓縮並提升色彩保存效果"""
    x = (x * 127.5 + 127.5).clamp(0, 255).to(torch.uint8).cpu()
    compressed_images = []
    for img in x:
        pil_img = torchvision.transforms.ToPILImage()(img)
        buffer = io.BytesIO()
        # 使用4:4:4色度採樣保持更好的色彩信息，除非質量非常低
        subsampling = "4:4:4" if quality > 30 else "4:2:0"
        pil_img.save(buffer, format="JPEG", quality=quality, subsampling=subsampling)
        buffer.seek(0)
        compressed_img = Image.open(buffer)
        compressed_tensor = torchvision.transforms.ToTensor()(compressed_img)
        compressed_images.append(compressed_tensor)
    return torch.stack(compressed_images).to(device).sub(0.5).div(0.5)

# 色彩損失函數
def color_loss(x, y):
    """計算色彩損失，著重在顏色變化而非亮度"""
    # 轉換到YCbCr空間的近似
    x_rgb = (x * 0.5 + 0.5).clamp(0, 1)
    y_rgb = (y * 0.5 + 0.5).clamp(0, 1)
    
    # 計算色度通道的差異，給予更高的權重
    color_diff = torch.abs(x_rgb - y_rgb)
    
    # 不同通道的權重 - 給色彩通道更高權重
    r_weight, g_weight, b_weight = 0.25, 0.5, 0.25
    weighted_diff = color_diff[:, 0:1] * r_weight + color_diff[:, 1:2] * g_weight + color_diff[:, 2:3] * b_weight
    
    return weighted_diff.mean()

# 時間嵌入模組
class TimeEmbedding(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        self.proj = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.SiLU(),
            nn.Linear(dim * 4, dim)
        )
        
    def forward(self, t):
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return self.proj(emb)

# 殘差注意力模塊
class ResAttnBlock(nn.Module):
    def __init__(self, in_c, out_c, time_dim, dropout=0.1):
        super().__init__()
        # 確保組數能被通道數整除
        num_groups = min(8, in_c)
        while in_c % num_groups != 0 and num_groups > 1:
            num_groups -= 1
            
        self.norm1 = nn.GroupNorm(num_groups, in_c)
        self.conv1 = nn.Conv2d(in_c, out_c, 3, padding=1)
        self.time_proj = nn.Linear(time_dim, out_c)
        
        # 調整 out_c 的組數
        num_groups_out = min(8, out_c)
        while out_c % num_groups_out != 0 and num_groups_out > 1:
            num_groups_out -= 1
            
        self.norm2 = nn.GroupNorm(num_groups_out, out_c)
        self.conv2 = nn.Conv2d(out_c, out_c, 3, padding=1)
        self.attn = nn.MultiheadAttention(out_c, 4, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        
        self.shortcut = nn.Conv2d(in_c, out_c, 1) if in_c != out_c else nn.Identity()
        
    def forward(self, x, t_emb):
        h = self.norm1(x)
        h = self.conv1(h)
        
        # 加入時間編碼
        t = self.time_proj(t_emb)[..., None, None]
        h = h + t
        
        h = self.norm2(h)
        h = self.conv2(F.silu(h))
        
        # 應用自注意力機制
        b, c, hh, ww = h.shape
        h_attn = h.view(b, c, -1).permute(0, 2, 1)
        h_attn, _ = self.attn(h_attn, h_attn, h_attn)
        h_attn = h_attn.permute(0, 2, 1).view(b, c, hh, ww)
        
        return self.shortcut(x) + self.dropout(h_attn)

# 改進的UNet架構
class JPEGDiffusionModel(nn.Module):
    def __init__(self):
        super().__init__()
        time_dim = 256
        self.time_embed = TimeEmbedding(time_dim)
        
        # 下采样路径 - 增强通道数以提高色彩保存能力
        self.down1 = ResAttnBlock(3, 64, time_dim)
        self.down2 = ResAttnBlock(64, 128, time_dim)
        self.down3 = ResAttnBlock(128, 256, time_dim)
        self.down4 = ResAttnBlock(256, 512, time_dim)
        self.down5 = ResAttnBlock(512, 512, time_dim)
        self.pool = nn.MaxPool2d(2)
        
        # 瓶颈层
        self.bottleneck = nn.Sequential(
            ResAttnBlock(512, 1024, time_dim),
            ResAttnBlock(1024, 1024, time_dim),
            ResAttnBlock(1024, 512, time_dim)
        )
        
        # 上采样路径
        self.up1 = ResAttnBlock(1024, 512, time_dim)
        self.up2 = ResAttnBlock(512 + 512, 256, time_dim)
        self.up3 = ResAttnBlock(256 + 256, 128, time_dim)
        self.up4 = ResAttnBlock(128 + 128, 64, time_dim)
        self.up5 = ResAttnBlock(64 + 64, 64, time_dim)
        
        # 输出层 - 使用1x1卷积保留空间色彩相关性
        self.out_conv = nn.Conv2d(64, 3, 1)
        
    def forward(self, x, t):
        t_emb = self.time_embed(t)
        
        # 下采样
        d1 = self.down1(x, t_emb)  # 32x32
        d2 = self.down2(self.pool(d1), t_emb)  # 16x16
        d3 = self.down3(self.pool(d2), t_emb)  # 8x8
        d4 = self.down4(self.pool(d3), t_emb)  # 4x4
        d5 = self.down5(self.pool(d4), t_emb)  # 2x2
        
        # 瓶颈层
        b = self.bottleneck[0](self.pool(d5), t_emb)
        b = self.bottleneck[1](b, t_emb)
        b = self.bottleneck[2](b, t_emb)
        
        # 上采样 - 使用双线性上采样避免棋盘格效应
        u1 = self.up1(torch.cat([F.interpolate(b, scale_factor=2, mode='bilinear', align_corners=False), d5], dim=1), t_emb)
        u2 = self.up2(torch.cat([F.interpolate(u1, scale_factor=2, mode='bilinear', align_corners=False), d4], dim=1), t_emb)
        u3 = self.up3(torch.cat([F.interpolate(u2, scale_factor=2, mode='bilinear', align_corners=False), d3], dim=1), t_emb)
        u4 = self.up4(torch.cat([F.interpolate(u3, scale_factor=2, mode='bilinear', align_corners=False), d2], dim=1), t_emb)
        u5 = self.up5(torch.cat([F.interpolate(u4, scale_factor=2, mode='bilinear', align_corners=False), d1], dim=1), t_emb)
        
        return self.out_conv(u5)

# 初始化模型
model = JPEGDiffusionModel().to(device)
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

# 設定優化器 - 使用AdamW配合學習率調整
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5, betas=(0.9, 0.99))
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=2)
loss_fn = nn.HuberLoss(reduction='mean', delta=1.0)  # 使用Huber損失，對於非高斯噪聲更穩健

# 設定JPEG擴散參數
num_timesteps = 100
# 根據論文，使用余弦調度的噪聲
betas = torch.linspace(1e-4, 0.02, num_timesteps).to(device)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)

# 高斯混合模型採樣器
class GaussianMixtureSampler:
    def __init__(self, model):
        self.model = model
        
    def optimize_mixture_params(self, x_t, pred_noise, t_step, t_next):
        """優化高斯混合模型參數"""
        # 預測的x0
        x0_pred = x_t + pred_noise
        
        # 使用預測的x0計算雙峰高斯分布的均值
        # 第一個均值 - 更傾向於保持原始預測
        mu1 = x0_pred * 0.9 + x_t * 0.1
        
        # 第二個均值 - 更傾向於向原始圖像方向移動
        mu2 = x0_pred * 1.1 - x_t * 0.1
        
        # 估算標準差 - 隨時間變化
        # 接近結束時使用較小的標準差
        time_weight = t_step / num_timesteps
        sigma_base = 0.15 * time_weight
        
        return mu1, mu2, sigma_base
        
    def sample(self, x_t, steps=100, guidance_scale=1.0):
        """使用高斯混合模型進行採樣"""
        self.model.eval()
        with torch.no_grad():
            # 從給定的噪聲圖像開始
            for i in tqdm(range(steps-1, -1, -1), desc="Sampling"):
                t = torch.full((x_t.size(0),), i, device=device).float() / num_timesteps
                
                # 獲取噪聲預測
                pred_noise = self.model(x_t, t)
                
                if i > 0:
                    # 計算高斯混合模型參數
                    mu1, mu2, sigma = self.optimize_mixture_params(x_t, pred_noise, i, i-1)
                    
                    # 隨機選擇使用哪個高斯分量
                    if random.random() < 0.33:  # 1/3概率使用第一個均值
                        next_mean = mu1
                    else:  # 2/3概率使用第二個均值
                        next_mean = mu2
                    
                    # 隨機性逐漸減少，接近原始圖像時幾乎無隨機性
                    noise_scale = sigma * (1.0 - (steps - i) / steps) * guidance_scale
                    
                    # 下一步
                    x_t = next_mean + torch.randn_like(x_t) * noise_scale
                else:
                    # 最後一步直接使用預測的原始圖像
                    x_t = x_t + pred_noise
        
        return x_t

# 改進的訓練函數
def train_epoch(model, loader, epoch):
    model.train()
    total_loss = 0
    color_loss_total = 0
    
    for x0, _ in tqdm(loader, desc=f"Training Epoch {epoch+1}"):
        x0 = x0.to(device)
        b = x0.size(0)
        
        # 使用結構化的質量分布
        # 確保模型見到各種質量水平的樣本，尤其是高質量
        if random.random() < 0.4 + min(0.4, epoch * 0.01):  # 隨著訓練進行增加高質量樣本比例
            # 高質量壓縮
            quality_range = (70, 100)
        elif random.random() < 0.6:
            # 中等質量壓縮
            quality_range = (40, 70)
        else:
            # 低質量壓縮
            quality_range = (5, 40)
            
        # 隨機選擇時間步
        t = torch.randint(1, num_timesteps, (b,), device=device).long()
        
        # 將時間步映射到質量因子，加入範圍控制
        min_q, max_q = quality_range
        quality = torch.clamp(min_q + (max_q - min_q) * (1 - t.float() / num_timesteps), 1, 100).cpu().numpy()
        
        # 對每個樣本使用不同質量進行JPEG壓縮
        xt = torch.stack([jpeg_compress(x0[i:i+1], int(q)) for i, q in enumerate(quality)]).squeeze()
        
        # 計算噪聲 (x0 - xt)
        noise = x0 - xt
        
        # 模型預測噪聲
        pred_noise = model(xt, t.float()/num_timesteps)
        
        # 計算損失 - 組合標準損失和色彩損失
        main_loss = loss_fn(pred_noise, noise)
        col_loss = color_loss(x0, xt + pred_noise)
        
        # 隨著訓練進行，逐漸增加色彩損失的權重
        color_weight = min(1.0, 0.2 + epoch * 0.02)
        loss = main_loss + color_weight * col_loss
        
        # 反向傳播更新
        optimizer.zero_grad()
        loss.backward()
        # 梯度裁剪防止爆炸
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += main_loss.item()
        color_loss_total += col_loss.item()
    
    avg_loss = total_loss / len(loader)
    avg_color_loss = color_loss_total / len(loader)
    print(f"Epoch {epoch+1} - Avg Loss: {avg_loss:.5f}, Color Loss: {avg_color_loss:.5f}, LR: {optimizer.param_groups[0]['lr']:.2e}")
    return avg_loss, avg_color_loss

# 驗證函數
def validate(model, loader, epoch):
    model.eval()
    total_loss = 0
    color_loss_total = 0
    
    with torch.no_grad():
        for x0, _ in tqdm(loader, desc=f"Validating Epoch {epoch+1}"):
            x0 = x0.to(device)
            b = x0.size(0)
            
            # 固定質量用於評估
            quality = torch.randint(10, 90, (b,)).cpu().numpy()
            xt = torch.stack([jpeg_compress(x0[i:i+1], int(q)) for i, q in enumerate(quality)]).squeeze()
            
            # 使用中間時間步進行評估
            t = torch.full((b,), num_timesteps//2, device=device).long()
            
            # 計算噪聲
            noise = x0 - xt
            
            # 模型預測噪聲
            pred_noise = model(xt, t.float()/num_timesteps)
            
            # 計算損失
            main_loss = loss_fn(pred_noise, noise)
            col_loss = color_loss(x0, xt + pred_noise)
            
            total_loss += main_loss.item()
            color_loss_total += col_loss.item()
            
    avg_loss = total_loss / len(loader)
    avg_color_loss = color_loss_total / len(loader)
    print(f"Validation - Avg Loss: {avg_loss:.5f}, Color Loss: {avg_color_loss:.5f}")
    
    # 保存一些復原效果樣本用於視覺檢查
    if epoch % 5 == 0:
        with torch.no_grad():
            x0, _ = next(iter(test_dataloader))
            x0 = x0.to(device)
            
            # 選擇不同的質量級別進行可視化
            qualities = [10, 30, 50, 70]
            plt.figure(figsize=(len(qualities)*3+3, 4))
            
            # 顯示原始圖像
            plt.subplot(1, len(qualities)+1, 1)
            plt.imshow(x0[0].cpu().permute(1,2,0)*0.5+0.5)
            plt.title("Original")
            plt.axis('off')
            
            # 顯示不同質量的JPEG壓縮和復原效果
            for i, q in enumerate(qualities):
                xt = jpeg_compress(x0, q)
                
                # 設定初始時間步基於質量
                t_step = int((100 - q) / 100 * num_timesteps)
                t_tensor = torch.full((1,), t_step, device=device).float() / num_timesteps
                
                # 預測噪聲
                pred_noise = model(xt, t_tensor)
                restored = xt + pred_noise
                
                plt.subplot(1, len(qualities)+1, i+2)
                plt.imshow(torch.cat([
                    xt[0].cpu().permute(1,2,0),
                    restored[0].cpu().permute(1,2,0)
                ], dim=1)*0.5+0.5)
                plt.title(f"Q{q}: JPEG vs Restored")
                plt.axis('off')
                
            plt.tight_layout()
            plt.savefig(f'validation_epoch_{epoch}.png')
            plt.close()
    
    return avg_loss, avg_color_loss

# 測試復原效果，使用高斯混合模型採樣器
def test_restoration(model, quality_levels=[10, 30, 50, 70]):
    # 初始化採樣器
    gmm_sampler = GaussianMixtureSampler(model)
    model.eval()
    
    with torch.no_grad():
        x0, _ = next(iter(test_dataloader))
        x0 = x0.to(device)
        
        plt.figure(figsize=(len(quality_levels)*3+3, 6))
        
        # 顯示原始圖像
        plt.subplot(2, len(quality_levels)+1, 1)
        plt.imshow(x0[0].cpu().permute(1,2,0)*0.5+0.5)
        plt.title("Original")
        plt.axis('off')
        
        # 為每個質量級別顯示JPEG和復原效果
        for i, q in enumerate(quality_levels):
            # JPEG壓縮
            xt = jpeg_compress(x0, q)
            
            # 設定初始時間步基於質量
            init_t = int((100 - q) / 100 * num_timesteps)
            
            # 使用GMM採樣器進行復原
            restored_gmm = gmm_sampler.sample(xt, steps=init_t+1)
            
            # 顯示JPEG壓縮結果
            plt.subplot(2, len(quality_levels)+1, i+2)
            plt.imshow(xt[0].cpu().permute(1,2,0)*0.5+0.5)
            plt.title(f"JPEG Q{q}")
            plt.axis('off')
            
            # 顯示復原結果
            plt.subplot(2, len(quality_levels)+1, len(quality_levels)+i+2)
            plt.imshow(restored_gmm[0].cpu().permute(1,2,0)*0.5+0.5)
            plt.title(f"Restored Q{q}")
            plt.axis('off')
        
        plt.tight_layout()
        plt.savefig('final_restoration_results.png')
        plt.close()
        
        # 計算和顯示復原指標
        metrics = {}
        for q in quality_levels:
            xt = jpeg_compress(x0, q)
            init_t = int((100 - q) / 100 * num_timesteps)
            restored_gmm = gmm_sampler.sample(xt, steps=init_t+1)
            
            # 計算PSNR (原始圖像與復原圖像)
            mse = F.mse_loss(restored_gmm, x0).item()
            psnr = 10 * math.log10(1.0 / mse)
            
            # 計算色彩保留指標
            col_loss = color_loss(restored_gmm, x0).item()
            
            metrics[q] = {
                'PSNR': psnr,
                'Color_Loss': col_loss
            }
        
        # 顯示指標
        print("\nRestoration Quality Metrics:")
        print("-" * 50)
        print(f"{'Quality':<10}{'PSNR (dB)':<15}{'Color Loss':<15}")
        print("-" * 50)
        for q in quality_levels:
            print(f"{q:<10}{metrics[q]['PSNR']:<15.2f}{metrics[q]['Color_Loss']:<15.5f}")
        
        return metrics

# 訓練循環
def train_model(epochs=100, patience=10):
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    color_losses = []
    patience_counter = 0
    
    for epoch in range(epochs):
        # 訓練一個周期
        train_loss, train_color_loss = train_epoch(model, train_dataloader, epoch)
        train_losses.append(train_loss)
        
        # 驗證一個周期
        val_loss, val_color_loss = validate(model, valid_dataloader, epoch)
        val_losses.append(val_loss)
        color_losses.append(val_color_loss)
        
        # 更新學習率
        scheduler.step()
        
        # 檢查是否需要保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'color_loss': val_color_loss
            }, f"best_jpeg_diffusion.pth")
            print(f"New best model saved with val loss {val_loss:.5f} and color loss {val_color_loss:.5f}")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping after {epoch+1} epochs!")
                break
        
        # 繪製訓練曲線
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Val Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(color_losses, label='Color Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Color Loss')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_curves.png')
        plt.close()
    
    print("Training completed!")
    
    # 載入最佳模型
    checkpoint = torch.load("best_jpeg_diffusion.pth")
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded best model from epoch {checkpoint['epoch']+1} with val loss {checkpoint['val_loss']:.5f}")
    
    # 測試復原效果
    test_restoration(model)

# 執行訓練
if __name__ == "__main__":
    train_model(epochs=500)

Using device: cuda
Files already downloaded and verified
Total parameters: 77,836,105


Training Epoch 1: 100%|██████████| 313/313 [00:49<00:00,  6.33it/s]


Epoch 1 - Avg Loss: 0.00285, Color Loss: 0.02435, LR: 1.00e-04


Validating Epoch 1: 100%|██████████| 40/40 [00:03<00:00, 12.48it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Validation - Avg Loss: 0.00303, Color Loss: 0.02621
New best model saved with val loss 0.00303 and color loss 0.02621


...

Epoch 86 - Avg Loss: 0.00096, Color Loss: 0.01340, LR: 5.45e-06


Validating Epoch 86: 100%|██████████| 40/40 [00:03<00:00, 12.65it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Validation - Avg Loss: 0.00238, Color Loss: 0.02242
Early stopping after 86 epochs!
Training completed!
Loaded best model from epoch 76 with val loss 0.00225


Sampling: 100%|██████████| 91/91 [00:00<00:00, 403.71it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Sampling: 100%|██████████| 71/71 [00:00<00:00, 419.40it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Sampling: 100%|██████████| 51/51 [00:00<00:00, 415.15it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Sampling: 100%|██████████| 31/31 [00:00<00:00, 416.83it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Sampling: 100%|██████████| 31/31 [00:00<00:00, 416.62it/s]


Restoration Quality Metrics:
--------------------------------------------------
Quality   PSNR (dB)      Color Loss     
--------------------------------------------------
10        -26.86         0.36525        
30        -23.68         0.22981        
50        11.22          0.07338        
70        17.94          0.04242        


In [5]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from PIL import Image
import io
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
import math
import os
from tqdm import tqdm
import numpy as np
from pytorch_msssim import ssim
import lpips

# 設備設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 資料預處理
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 下載 CIFAR-10 測試資料集
test_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# JPEG 壓縮函數
def jpeg_compress(x, quality):
    """執行JPEG壓縮並提升色彩保存效果"""
    x = (x * 127.5 + 127.5).clamp(0, 255).to(torch.uint8).cpu()
    compressed_images = []
    for img in x:
        pil_img = torchvision.transforms.ToPILImage()(img)
        buffer = io.BytesIO()
        # 使用4:4:4色度採樣保持更好的色彩信息，除非質量非常低
        subsampling = "4:4:4" if quality > 30 else "4:2:0"
        pil_img.save(buffer, format="JPEG", quality=quality, subsampling=subsampling)
        buffer.seek(0)
        compressed_img = Image.open(buffer)
        compressed_tensor = torchvision.transforms.ToTensor()(compressed_img)
        compressed_images.append(compressed_tensor)
    return torch.stack(compressed_images).to(device).sub(0.5).div(0.5)

# 時間嵌入模組
class TimeEmbedding(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        self.proj = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.SiLU(),
            nn.Linear(dim * 4, dim)
        )
        
    def forward(self, t):
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return self.proj(emb)

# 殘差注意力模塊
class ResAttnBlock(nn.Module):
    def __init__(self, in_c, out_c, time_dim, dropout=0.1):
        super().__init__()
        # 確保組數能被通道數整除
        num_groups = min(8, in_c)
        while in_c % num_groups != 0 and num_groups > 1:
            num_groups -= 1
            
        self.norm1 = nn.GroupNorm(num_groups, in_c)
        self.conv1 = nn.Conv2d(in_c, out_c, 3, padding=1)
        self.time_proj = nn.Linear(time_dim, out_c)
        
        # 調整 out_c 的組數
        num_groups_out = min(8, out_c)
        while out_c % num_groups_out != 0 and num_groups_out > 1:
            num_groups_out -= 1
            
        self.norm2 = nn.GroupNorm(num_groups_out, out_c)
        self.conv2 = nn.Conv2d(out_c, out_c, 3, padding=1)
        self.attn = nn.MultiheadAttention(out_c, 4, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        
        self.shortcut = nn.Conv2d(in_c, out_c, 1) if in_c != out_c else nn.Identity()
        
    def forward(self, x, t_emb):
        h = self.norm1(x)
        h = self.conv1(h)
        
        # 加入時間編碼
        t = self.time_proj(t_emb)[..., None, None]
        h = h + t
        
        h = self.norm2(h)
        h = self.conv2(F.silu(h))
        
        # 應用自注意力機制
        b, c, hh, ww = h.shape
        h_attn = h.view(b, c, -1).permute(0, 2, 1)
        h_attn, _ = self.attn(h_attn, h_attn, h_attn)
        h_attn = h_attn.permute(0, 2, 1).view(b, c, hh, ww)
        
        return self.shortcut(x) + self.dropout(h_attn)

# 改進的UNet架構
class JPEGDiffusionModel(nn.Module):
    def __init__(self):
        super().__init__()
        time_dim = 256
        self.time_embed = TimeEmbedding(time_dim)
        
        # 下采样路径 - 增强通道数以提高色彩保存能力
        self.down1 = ResAttnBlock(3, 64, time_dim)
        self.down2 = ResAttnBlock(64, 128, time_dim)
        self.down3 = ResAttnBlock(128, 256, time_dim)
        self.down4 = ResAttnBlock(256, 512, time_dim)
        self.down5 = ResAttnBlock(512, 512, time_dim)
        self.pool = nn.MaxPool2d(2)
        
        # 瓶颈层
        self.bottleneck = nn.Sequential(
            ResAttnBlock(512, 1024, time_dim),
            ResAttnBlock(1024, 1024, time_dim),
            ResAttnBlock(1024, 512, time_dim)
        )
        
        # 上采样路径
        self.up1 = ResAttnBlock(1024, 512, time_dim)
        self.up2 = ResAttnBlock(512 + 512, 256, time_dim)
        self.up3 = ResAttnBlock(256 + 256, 128, time_dim)
        self.up4 = ResAttnBlock(128 + 128, 64, time_dim)
        self.up5 = ResAttnBlock(64 + 64, 64, time_dim)
        
        # 输出层 - 使用1x1卷积保留空间色彩相关性
        self.out_conv = nn.Conv2d(64, 3, 1)
        
    def forward(self, x, t):
        t_emb = self.time_embed(t)
        
        # 下采样
        d1 = self.down1(x, t_emb)  # 32x32
        d2 = self.down2(self.pool(d1), t_emb)  # 16x16
        d3 = self.down3(self.pool(d2), t_emb)  # 8x8
        d4 = self.down4(self.pool(d3), t_emb)  # 4x4
        d5 = self.down5(self.pool(d4), t_emb)  # 2x2
        
        # 瓶颈层
        b = self.bottleneck[0](self.pool(d5), t_emb)
        b = self.bottleneck[1](b, t_emb)
        b = self.bottleneck[2](b, t_emb)
        
        # 上采样 - 使用双线性上采样避免棋盘格效应
        u1 = self.up1(torch.cat([F.interpolate(b, scale_factor=2, mode='bilinear', align_corners=False), d5], dim=1), t_emb)
        u2 = self.up2(torch.cat([F.interpolate(u1, scale_factor=2, mode='bilinear', align_corners=False), d4], dim=1), t_emb)
        u3 = self.up3(torch.cat([F.interpolate(u2, scale_factor=2, mode='bilinear', align_corners=False), d3], dim=1), t_emb)
        u4 = self.up4(torch.cat([F.interpolate(u3, scale_factor=2, mode='bilinear', align_corners=False), d2], dim=1), t_emb)
        u5 = self.up5(torch.cat([F.interpolate(u4, scale_factor=2, mode='bilinear', align_corners=False), d1], dim=1), t_emb)
        
        return self.out_conv(u5)

# 初始化模型
model = JPEGDiffusionModel().to(device)
print(f"總參數數量: {sum(p.numel() for p in model.parameters()):,}")

# 高斯混合模型採樣器
class GaussianMixtureSampler:
    def __init__(self, model):
        self.model = model
        
    def optimize_mixture_params(self, x_t, pred_noise, t_step, t_next):
        """優化高斯混合模型參數"""
        # 預測的x0
        x0_pred = x_t + pred_noise
        
        # 使用預測的x0計算雙峰高斯分布的均值
        # 第一個均值 - 更傾向於保持原始預測
        mu1 = x0_pred * 0.9 + x_t * 0.1
        
        # 第二個均值 - 更傾向於向原始圖像方向移動
        mu2 = x0_pred * 1.1 - x_t * 0.1
        
        # 估算標準差 - 隨時間變化
        # 接近結束時使用較小的標準差
        time_weight = t_step / num_timesteps
        sigma_base = 0.15 * time_weight
        
        return mu1, mu2, sigma_base
        
    def sample(self, x_t, steps=100, guidance_scale=1.0):
        """使用高斯混合模型進行採樣"""
        self.model.eval()
        with torch.no_grad():
            # 從給定的噪聲圖像開始
            for i in tqdm(range(steps-1, -1, -1), desc="Sampling"):
                t = torch.full((x_t.size(0),), i, device=device).float() / num_timesteps
                
                # 獲取噪聲預測
                pred_noise = self.model(x_t, t)
                
                if i > 0:
                    # 計算高斯混合模型參數
                    mu1, mu2, sigma = self.optimize_mixture_params(x_t, pred_noise, i, i-1)
                    
                    # 隨機選擇使用哪個高斯分量
                    if random.random() < 0.33:  # 1/3概率使用第一個均值
                        next_mean = mu1
                    else:  # 2/3概率使用第二個均值
                        next_mean = mu2
                    
                    # 隨機性逐漸減少，接近原始圖像時幾乎無隨機性
                    noise_scale = sigma * (1.0 - (steps - i) / steps) * guidance_scale
                    
                    # 下一步
                    x_t = next_mean + torch.randn_like(x_t) * noise_scale
                else:
                    # 最後一步直接使用預測的原始圖像
                    x_t = x_t + pred_noise
        
        return x_t

# 設定JPEG擴散參數
num_timesteps = 100
# 根據論文，使用余弦調度的噪聲
betas = torch.linspace(1e-4, 0.02, num_timesteps).to(device)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)

# 嘗試加載模型權重
try:
    checkpoint = torch.load("best_jpeg_diffusion.pth")
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"成功載入模型權重，來自 epoch {checkpoint['epoch']+1}")
    else:
        model.load_state_dict(checkpoint)
        print("成功載入模型權重!")
    model.eval()
except Exception as e:
    print(f"載入模型失敗: {e}")
    print("請確保 'best_jpeg_diffusion.pth' 文件存在且有效")
    exit()

# 初始化採樣器
sampler = GaussianMixtureSampler(model)

# 初始化LPIPS函數
lpips_fn = lpips.LPIPS(net='alex').to(device)

# 計算PSNR
def calculate_psnr(img1, img2):
    # 確保值域在[0, 1]之間
    img1 = (img1 * 0.5 + 0.5).clamp(0, 1)
    img2 = (img2 * 0.5 + 0.5).clamp(0, 1)
    
    mse = F.mse_loss(img1, img2)
    if mse == 0:
        return float('inf')
    return 10 * torch.log10(1.0 / mse)

# 創建保存結果的資料夾
os.makedirs("inference_results", exist_ok=True)

# 運行推理與評估
def run_inference(num_samples=10, qualities=[10, 30, 50, 70]):
    """執行推理與評估"""
    psnr_compressed_total = {q: 0.0 for q in qualities}
    psnr_restored_total = {q: 0.0 for q in qualities}
    ssim_compressed_total = {q: 0.0 for q in qualities}
    ssim_restored_total = {q: 0.0 for q in qualities}
    lpips_compressed_total = {q: 0.0 for q in qualities}
    lpips_restored_total = {q: 0.0 for q in qualities}
    
    for i, (x0, _) in enumerate(test_dataloader):
        if i >= num_samples:
            break
        
        x0 = x0.to(device)
        
        for quality in qualities:
            # JPEG壓縮
            x_compressed = jpeg_compress(x0, quality)
            
            # 設定初始時間步基於質量
            init_t = int((100 - quality) / 100 * num_timesteps)
            
            # 使用高斯混合模型採樣器進行復原
            x_restored = sampler.sample(x_compressed, steps=init_t+1, guidance_scale=0.8)
            
            # 轉換到[0,1]範圍用於計算指標
            x0_01 = (x0 * 0.5 + 0.5).clamp(0, 1)
            x_compressed_01 = (x_compressed * 0.5 + 0.5).clamp(0, 1)
            x_restored_01 = (x_restored * 0.5 + 0.5).clamp(0, 1)
            
            # 計算PSNR
            psnr_comp = calculate_psnr(x0, x_compressed).item()
            psnr_rest = calculate_psnr(x0, x_restored).item()
            
            # 計算SSIM
            ssim_comp = ssim(x0_01, x_compressed_01, data_range=1.0).item()
            ssim_rest = ssim(x0_01, x_restored_01, data_range=1.0).item()
            
            # 計算LPIPS (較低的值表示更好的感知相似度)
            lpips_comp = lpips_fn(x0 * 2, x_compressed * 2).item()  # LPIPS期望[-1,1]範圍
            lpips_rest = lpips_fn(x0 * 2, x_restored * 2).item()
            
            # 累計指標
            psnr_compressed_total[quality] += psnr_comp
            psnr_restored_total[quality] += psnr_rest
            ssim_compressed_total[quality] += ssim_comp
            ssim_restored_total[quality] += ssim_rest
            lpips_compressed_total[quality] += lpips_comp
            lpips_restored_total[quality] += lpips_rest
            
            print(f"影像 {i+1}, 質量 {quality}: "
                  f"PSNR (壓縮/恢復): {psnr_comp:.2f}dB/{psnr_rest:.2f}dB, "
                  f"SSIM (壓縮/恢復): {ssim_comp:.4f}/{ssim_rest:.4f}, "
                  f"LPIPS (壓縮/恢復): {lpips_comp:.4f}/{lpips_rest:.4f}")
            
            # 每個質量級別的第一張圖片儲存用於視覺比較
            if i == 0:
                fig, axes = plt.subplots(1, 3, figsize=(15, 5))
                
                axes[0].imshow(x0_01[0].cpu().permute(1, 2, 0))
                axes[0].set_title("原始圖像")
                axes[0].axis("off")
                
                axes[1].imshow(x_compressed_01[0].cpu().permute(1, 2, 0))
                axes[1].set_title(f"JPEG Q{quality}\nPSNR: {psnr_comp:.2f}dB\nSSIM: {ssim_comp:.4f}\nLPIPS: {lpips_comp:.4f}")
                axes[1].axis("off")
                
                axes[2].imshow(x_restored_01[0].cpu().permute(1, 2, 0))
                axes[2].set_title(f"恢復圖像\nPSNR: {psnr_rest:.2f}dB\nSSIM: {ssim_rest:.4f}\nLPIPS: {lpips_rest:.4f}")
                axes[2].axis("off")
                
                plt.tight_layout()
                plt.savefig(f"inference_results/quality_{quality}_comparison.png", dpi=150)
                plt.close()
                
        # 顯示一個完整的例子(所有質量級別)
        if i == 0:
            n_cols = len(qualities) + 1
            fig, axes = plt.subplots(2, n_cols, figsize=(n_cols*4, 8))
            
            # 第一行顯示原圖和各種品質的壓縮圖
            axes[0, 0].imshow((x0[0].cpu().permute(1, 2, 0) * 0.5 + 0.5).clamp(0, 1))
            axes[0, 0].set_title("原始圖像")
            axes[0, 0].axis("off")
            
            for j, quality in enumerate(qualities):
                x_compressed = jpeg_compress(x0, quality)
                axes[0, j+1].imshow((x_compressed[0].cpu().permute(1, 2, 0) * 0.5 + 0.5).clamp(0, 1))
                axes[0, j+1].set_title(f"JPEG Q{quality}")
                axes[0, j+1].axis("off")
            
            # 第二行顯示原圖和各種品質的復原圖
            axes[1, 0].imshow((x0[0].cpu().permute(1, 2, 0) * 0.5 + 0.5).clamp(0, 1))
            axes[1, 0].set_title("原始圖像")
            axes[1, 0].axis("off")
            
            for j, quality in enumerate(qualities):
                x_compressed = jpeg_compress(x0, quality)
                init_t = int((100 - quality) / 100 * num_timesteps)
                x_restored = sampler.sample(x_compressed, steps=init_t+1)
                
                axes[1, j+1].imshow((x_restored[0].cpu().permute(1, 2, 0) * 0.5 + 0.5).clamp(0, 1))
                axes[1, j+1].set_title(f"從Q{quality}恢復")
                axes[1, j+1].axis("off")
            
            plt.tight_layout()
            plt.savefig(f"inference_results/all_qualities_comparison.png", dpi=150)
            plt.close()
    
    # 計算平均指標
    for quality in qualities:
        avg_psnr_comp = psnr_compressed_total[quality] / num_samples
        avg_psnr_rest = psnr_restored_total[quality] / num_samples
        avg_ssim_comp = ssim_compressed_total[quality] / num_samples
        avg_ssim_rest = ssim_restored_total[quality] / num_samples
        avg_lpips_comp = lpips_compressed_total[quality] / num_samples
        avg_lpips_rest = lpips_restored_total[quality] / num_samples
        
        psnr_gain = avg_psnr_rest - avg_psnr_comp
        ssim_gain = avg_ssim_rest - avg_ssim_comp
        lpips_improvement = avg_lpips_comp - avg_lpips_rest  # LPIPS越低越好
        
        print(f"\nJPEG 質量: {quality}")
        print(f"平均 PSNR (原始 vs 壓縮): {avg_psnr_comp:.2f} dB")
        print(f"平均 PSNR (原始 vs 恢復): {avg_psnr_rest:.2f} dB")
        print(f"PSNR 提升: {psnr_gain:.2f} dB")
        
        print(f"平均 SSIM (原始 vs 壓縮): {avg_ssim_comp:.4f}")
        print(f"平均 SSIM (原始 vs 恢復): {avg_ssim_rest:.4f}")
        print(f"SSIM 提升: {ssim_gain:.4f}")
        
        print(f"平均 LPIPS (原始 vs 壓縮): {avg_lpips_comp:.4f}")
        print(f"平均 LPIPS (原始 vs 恢復): {avg_lpips_rest:.4f}")
        print(f"LPIPS 改善: {lpips_improvement:.4f}")
    
    # 繪製比較圖
    plt.figure(figsize=(15, 6))
    
    # PSNR比較圖
    plt.subplot(1, 3, 1)
    x = np.arange(len(qualities))
    width = 0.35
    plt.bar(x - width/2, [psnr_compressed_total[q]/num_samples for q in qualities], width, label='壓縮')
    plt.bar(x + width/2, [psnr_restored_total[q]/num_samples for q in qualities], width, label='恢復')
    plt.xlabel('JPEG 質量')
    plt.ylabel('PSNR (dB)')
    plt.title('PSNR 比較')
    plt.xticks(x, qualities)
    plt.legend()
    
    # SSIM比較圖
    plt.subplot(1, 3, 2)
    plt.bar(x - width/2, [ssim_compressed_total[q]/num_samples for q in qualities], width, label='壓縮')
    plt.bar(x + width/2, [ssim_restored_total[q]/num_samples for q in qualities], width, label='恢復')
    plt.xlabel('JPEG 質量')
    plt.ylabel('SSIM')
    plt.title('SSIM 比較')
    plt.xticks(x, qualities)
    plt.legend()
    
    # LPIPS比較圖
    plt.subplot(1, 3, 3)
    plt.bar(x - width/2, [lpips_compressed_total[q]/num_samples for q in qualities], width, label='壓縮')
    plt.bar(x + width/2, [lpips_restored_total[q]/num_samples for q in qualities], width, label='恢復')
    plt.xlabel('JPEG 質量')
    plt.ylabel('LPIPS (越低越好)')
    plt.title('LPIPS 比較')
    plt.xticks(x, qualities)
    plt.legend()
    
    plt.tight_layout()
    plt.savefig("inference_results/metrics_comparison.png", dpi=150)
    plt.close()
    
    print(f"\n評估完成，結果已保存至 'inference_results' 目錄")

# 主函數
if __name__ == "__main__":
    # 設定要測試的樣本數量
    num_samples = 10
    
    # 設定要測試的JPEG質量級別
    qualities = [10, 30, 50, 70]
    
    # 運行推理
    run_inference(num_samples, qualities)


Using device: cuda
Files already downloaded and verified
總參數數量: 77,836,105
成功載入模型權重，來自 epoch 1
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


Sampling: 100%|██████████| 91/91 [00:00<00:00, 419.26it/s]
/tmp/ipykernel_3062/1484099668.py:344: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:344: UserWarning: Glyph 22987 (\N{CJK UNIFIED IDEOGRAPH-59CB}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:344: UserWarning: Glyph 22294 (\N{CJK UNIFIED IDEOGRAPH-5716}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:344: UserWarning: Glyph 20687 (\N{CJK UNIFIED IDEOGRAPH-50CF}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:344: UserWarning: Glyph 24674 (\N{CJK UNIFIED IDEOGRAPH-6062}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:344: UserWarning: Glyph 24489 (\N{CJK UNIFIED IDEOGRAPH-5FA9}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:345: UserWarning: Glyph 21407 (\N{CJK

影像 1, 質量 10: PSNR (壓縮/恢復): 22.53dB/6.36dB, SSIM (壓縮/恢復): 0.6776/0.0291, LPIPS (壓縮/恢復): 0.0395/0.2888


...

影像 10, 質量 50: PSNR (壓縮/恢復): 26.80dB/10.76dB, SSIM (壓縮/恢復): 0.9092/0.2485, LPIPS (壓縮/恢復): 0.0019/0.1665


Sampling: 100%|██████████| 31/31 [00:00<00:00, 431.72it/s]


影像 10, 質量 70: PSNR (壓縮/恢復): 28.82dB/16.62dB, SSIM (壓縮/恢復): 0.9360/0.5603, LPIPS (壓縮/恢復): 0.0006/0.0960

JPEG 質量: 10
平均 PSNR (原始 vs 壓縮): 24.03 dB
平均 PSNR (原始 vs 恢復): 6.18 dB
PSNR 提升: -17.86 dB
平均 SSIM (原始 vs 壓縮): 0.7434
平均 SSIM (原始 vs 恢復): 0.0249
SSIM 提升: -0.7185
平均 LPIPS (原始 vs 壓縮): 0.0480
平均 LPIPS (原始 vs 恢復): 0.3142
LPIPS 改善: -0.2663

JPEG 質量: 30
平均 PSNR (原始 vs 壓縮): 27.53 dB
平均 PSNR (原始 vs 恢復): 7.77 dB
PSNR 提升: -19.76 dB
平均 SSIM (原始 vs 壓縮): 0.8598
平均 SSIM (原始 vs 恢復): 0.0919
SSIM 提升: -0.7678
平均 LPIPS (原始 vs 壓縮): 0.0174
平均 LPIPS (原始 vs 恢復): 0.3009
LPIPS 改善: -0.2835

JPEG 質量: 50
平均 PSNR (原始 vs 壓縮): 30.20 dB
平均 PSNR (原始 vs 恢復): 11.55 dB
PSNR 提升: -18.64 dB
平均 SSIM (原始 vs 壓縮): 0.9104
平均 SSIM (原始 vs 恢復): 0.2411
SSIM 提升: -0.6693
平均 LPIPS (原始 vs 壓縮): 0.0054
平均 LPIPS (原始 vs 恢復): 0.2205
LPIPS 改善: -0.2151

JPEG 質量: 70
平均 PSNR (原始 vs 壓縮): 32.03 dB
平均 PSNR (原始 vs 恢復): 19.16 dB
PSNR 提升: -12.86 dB
平均 SSIM (原始 vs 壓縮): 0.9399
平均 SSIM (原始 vs 恢復): 0.5455
SSIM 提升: -0.3944
平均 LPIPS (原始 vs 壓縮): 0.0027
平均 LP

/tmp/ipykernel_3062/1484099668.py:443: UserWarning: Glyph 36074 (\N{CJK UNIFIED IDEOGRAPH-8CEA}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:443: UserWarning: Glyph 37327 (\N{CJK UNIFIED IDEOGRAPH-91CF}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:443: UserWarning: Glyph 27604 (\N{CJK UNIFIED IDEOGRAPH-6BD4}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:443: UserWarning: Glyph 36611 (\N{CJK UNIFIED IDEOGRAPH-8F03}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:443: UserWarning: Glyph 22739 (\N{CJK UNIFIED IDEOGRAPH-58D3}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:443: UserWarning: Glyph 32302 (\N{CJK UNIFIED IDEOGRAPH-7E2E}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_3062/1484099668.py:443: UserWarning: Glyph 24674 (\N{CJK UNIFIED IDEOGRAPH-6062}) missing from current font.
  plt.


評估完成，結果已保存至 'inference_results' 目錄
